Example copy:

1 Torchaudio load from S3:
client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
response = client.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
waveform, sample_rate = torchaudio.load(response['Body'])
plot_specgram(waveform, sample_rate, title="From S3")



Pre installatie
- sudo apt install libsndfile1-dev
- sudo apt install awscli

- aws configure sso (Met informatie geleverd door Nick)

- conda create --name humainr python=3.9
- pip install boto3
- pip install transformers"[speech, sentencepiece]"
- pip install datasets


Verschillende imports voor het programma

In [3]:
### General
import os
import torch
import uuid
import shutil
from io import BytesIO
import json

### s3 and DynamoDB
import boto3

### Transcibing trough ML
import torchaudio
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset, Audio


Configuratie

In [6]:

boto3.setup_default_session(profile_name='AWSIntroTraining-408122842185')
s3 = boto3.resource('s3')
bucketname = 'humainr-aws-intro-bucket'
bucket = s3.Bucket(bucketname)

sync_path = 'data/new/'
arch_path = 'data/archive/'
test_file = 'common_voice_en_38497561.mp3'

for my_bucket_object in bucket.objects.all():
    print(my_bucket_object.key)


common_voice_en_38497561 (copy).mp3
common_voice_en_38497561.mp3
test.json
humainr-aws-intro-bucket
common_voice_en_38497561.mp3


Verwerk de klaarstaande audiobestanden naar de AWS bucket

1 - Upload file naar S3
2 - Verplaats bestand van de sync_dir naar de arch_dir

In [17]:

print("Inhoud van de locale disk:")
for entry in os.listdir(sync_path):
    print(entry)
    if entry.endswith('.mp3'):
        bucket.upload_file(sync_path + entry, entry)
        shutil.move(sync_path + entry, arch_path + entry)
print("")

print("Inhoud van de s3 bucket:")
for my_bucket_object in bucket.objects.all():
    print(my_bucket_object.key)
    ### Volgende regel is voor debug om de s3 bucket geheel leeg te gooien.
    #my_bucket_object.delete()
print("")



Inhoud van de locale disk:
validation.txt

Inhoud van de s3 bucket:
common_voice_en_38497561 (copy).mp3
common_voice_en_38497561.mp3
test.json



Sectie voor de transcriptie:

1 - Zet bytestream om naar bruikbare data
2 - Pas de samplingrate aan
3 - Extraheer feautures
4 - Transcribeer de data

In [18]:

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

### Laad de file/files in de dataset die transcribed moet worden
#ds = load_dataset("audiofolder", data_files=[sync_path + test_file], split="train")
obj = s3.Object(bucket_name=bucketname, key=test_file)
response = obj.get()
audio_bytes = BytesIO(response['Body'].read())

### Zet de bytestream van de s3 opslag om naar een numberarray
waveform, sample_rate = torchaudio.load(audio_bytes, format="mp3")

### Zet de sampling rate om naar 16000 voor het getrainde model
waveform = torchaudio.functional.resample(waveform, orig_freq=sample_rate, new_freq=16000)
print(waveform)

### Genereer features uit de data
inputs = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt")

### [TODO] Uitzoeken wat deze functie precies doet.
generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

### [TODO] Omzetten prediction naar transcriptie.
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.encoder.embed_positions.weights', 'model.decoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[ 2.6299e-12,  9.4797e-12, -1.5897e-12,  ..., -3.0086e-04,
         -1.3465e-04, -3.2022e-05]])


Opslaan transcriptie
1 - Tijdelijk json bestand
2 - Upload naar de s3 bucket
3 - Verwijder tijdelijk bestand

In [19]:

with open("test.json", "w") as file:
    json.dump(transcription, file)

bucket.upload_file('test.json', 'test.json')

os.remove('test.json')

for my_bucket_object in bucket.objects.all():
    print(my_bucket_object.key)

print(uuid.uuid4())


common_voice_en_38497561 (copy).mp3
common_voice_en_38497561.mp3
test.json
9885e413-f37e-4131-bb7d-a666d2742ff7


Verwerk de transcriptie en metadata in DynamoDB

Verwerk de al verwerkte data naar een leesbare html pagina

Start html server op om verwerkte data te publiseren

[TODO] Pagina maken om data te uploaden en te gaan verwerken. (als er tijd over is)